# Chapter 1 Deducing Types

## Item 1: Understand template type deduction.

```cpp
template<typename T> void f(ParamType param);
f(expr); // deduce T and ParamType from expr
```

### Case 1: ParamType が参照もしくはポインタだが、ユニヴァーサル参照ではない

1. expr が参照型ならば、参照性(参照動作部分)を無視する。
2. expr の型を ParamType とパターンマッチングし、T を決定する。

In [1]:
template<typename T> void f(T& param) {
    std::cout << param << std::endl;
}

In [2]:
// T is int, param's type is int&

int x = 27;
f(x);

27


In [3]:
// T is const int,
// param's type is const int&

const int cx = x;
f(cx);

27


In [4]:
// T is const int,
// param's type is const int&

const int& rx = x;
f(rx);

27


In [5]:
template<typename T> void f_const(const T& param) {
    std::cout << param << std::endl;
}

In [6]:
// T is int, param's type is const int&

f_const(x)

27


In [7]:
// T is int, param's type is const int&

f_const(cx);

27


In [8]:
// T is int, param's type is const int&

f_const(rx);

27


In [9]:
template<typename T> void f_pointer(T* param) {
    std::cout << param << std::endl;
}

In [10]:
// T is int, param's type is int*

f_pointer(&x);

0x115432e4c


In [11]:
// T is const int,
// param's type is const int*

const int *px = &x;
f_pointer(px);

0x115432e4c


### Case 2: ParamType がユニヴァーサル参照である

- expr が左辺値ならば、T も ParamType も左辺値参照と推論される。これは 2 つの意味で特 殊である。まず、テンプレートの型推論で、T を参照として推論するのはこの場合だけであ る。もう 1 つは、ParamType の宣言には右辺値参照という形態をとりながら、推論される型 は左辺値参照となる点である。
- expr が右辺値の場合は、「通常の」規則が適用される(ケース 1)。

In [12]:
// param is now a universal reference

template<typename T> void f_universal(T&& param) {
    std::cout << param << std::endl;
}

In [13]:
// x is lvalue, so T is int&, // param's type is also int&

int y = 27;
f_universal(y);

27


In [14]:
// cx is lvalue, so T is const int&,
// param's type is also const int&

const int cy = y;
f_universal(cy);

27


In [15]:
// rx is lvalue, so T is const int&, 
// param's type is also const int&

const int& ry = y;
f_universal(ry);

27


In [16]:
// 27 is rvalue, so T is int,
// param's type is therefore int&&

f_universal(27);

27


### Case 3: ParamType がポインタでも参照でもない

ParamType がポインタでも参照でもなければ、値渡しとなる。

1. これまでと同様に、expr の型が参照であれば、参照性(参照動作部分)は無視される。
2. 参照性を無視した expr が const であれば、これも無視する。volatile であれば、同様にこ れも無視する(volatile オブジェクトは滅多に使用されない。使用されるのは一般にデバイスドライバを実装する場合に限られる。詳細は項目 40 を参照)。

In [17]:
template<typename T> void f_value(T param) {
    std::cout << param << std::endl;
}

In [18]:
// T's and param's types are both int

int z = 27;
f_value(z);

27


In [19]:
// T's and param's types are again both int

const int cz = z;
f_value(cz);

27


In [20]:
// T's and param's types are still both int

const int& rz = z;
f_value(rz);

27


In [21]:
// param's type is const char*

const char* const ptr = "Fun with pointers";
f_value(ptr)

Fun with pointers
